<a href="https://colab.research.google.com/github/hselino/complaint_analysis/blob/main/original_df_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.consumerfinance.gov/ccdb/complaints.csv.zip
!pip install langdetect

--2025-02-19 23:11:02--  https://files.consumerfinance.gov/ccdb/complaints.csv.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 18.154.206.56, 18.154.206.79, 18.154.206.102, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|18.154.206.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1045780187 (997M) [binary/octet-stream]
Saving to: ‘complaints.csv.zip’

complaints.csv.zip  100%[===================>] 997.33M  27.2MB/s    in 36s     

2025-02-19 23:11:39 (28.0 MB/s) - ‘complaints.csv.zip’ saved [1045780187/1045780187]



In [ ]:
!unzip /content/complaints.csv.zip

Archive:  /content/complaints.csv.zip
  inflating: complaints.csv          


In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv('/content/complaints.csv')
print("Original columns:", df.columns.tolist())

<ipython-input-2-c5c290fa1409>:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/complaints.csv')


Original columns: ['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Company public response', 'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Date sent to company', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID']


In [ ]:
columns_to_drop = [
    'Sub-product', 'Sub-issue', 'Company public response', 'ZIP code',
    'Tags', 'Consumer consent provided?', 'Submitted via',
    'Date sent to company', 'Timely response?','State','Complaint ID','Date received','Consumer disputed?'
]

# Drop the unnecessary columns (ignore errors if some columns are missing)
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [ ]:
df['Consumer complaint narrative'].isna().sum()

5325751

In [ ]:
df2 = df.copy()

In [ ]:
# Drrop missing values in the core text field
df2 = df2.dropna(subset=['Consumer complaint narrative'])
df2.reset_index(inplace=True, drop=True)

In [ ]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Replace sequences of two or more 'x' characters with "[redacted]"
    text = re.sub(r'x{2,}', '[redacted]', text)
    # Remove punctuation and non-alphanumeric characters (except spaces and square brackets)
    text = re.sub(r'[^a-z0-9\s\[\]]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply preprocessing to the complaint narratives
df2['processed_complaint'] = df2['Consumer complaint narrative'].apply(preprocess_text)


In [ ]:
df2.to_csv('/content/drive/MyDrive/prj/pr.csv', index=False)

In [ ]:
#df2 = pd.read_csv('/content/drive/MyDrive/prj/pr.csv')

before converting text into vector embeddings

In [ ]:
df2 = df2.drop_duplicates(subset=['processed_complaint'])

In [ ]:
'''from langdetect import detect
# Language Detection & Segregation
df2['language'] = df2['processed_complaint'].apply(lambda x: detect(x))

KeyboardInterrupt: 

In [ ]:
#Remove URLs, Emails, and Special Tokens
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')
EMAIL_PATTERN = re.compile(r'\S+@\S+')
def remove_noise(text):
    text = URL_PATTERN.sub('', text)
    text = EMAIL_PATTERN.sub('', text)
    return text

df2['processed_complaint'] = df2['processed_complaint'].apply(remove_noise)

In [ ]:
df2.to_csv('/content/drive/MyDrive/prj/pr2.csv',index = False)

In [ ]:
df2

,Product,Issue,Consumer complaint narrative,Company,Company response to consumer,processed_complaint
0,Credit reporting or other personal consumer re...,Incorrect information on your report,TransUnion was tasked with investigating and r...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with non-monetary relief,transunion was tasked with investigating and r...
1,Credit reporting or other personal consumer re...,Incorrect information on your report,"THESE ARE NOT MINE, I DID NOT NOR BENEFIT FROM...","TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with non-monetary relief,these are not mine i did not nor benefit from ...
2,Credit reporting or other personal consumer re...,Incorrect information on your report,"See attached documents XXXX XXXX, Consumer Soc...","TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with non-monetary relief,see attached documents [redacted] [redacted] c...
3,Credit reporting or other personal consumer re...,Incorrect information on your report,In accordance with the Fair Credit Reporting A...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with non-monetary relief,in accordance with the fair credit reporting a...
4,Credit reporting or other personal consumer re...,Problem with a company's investigation into an...,I have sent multiple complaints through CFPB a...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with non-monetary relief,i have sent multiple complaints through cfpb a...
...,...,...,...,...,...,...
2506642,"Credit reporting, credit repair services, or o...",Incorrect information on your report,I sent a letter on XX/XX/XXXX to 3 credit bure...,"EQUIFAX, INC.",Closed with explanation,i sent a letter on [redacted][redacted][redact...
2506643,"Credit reporting, credit repair services, or o...",Problem with a credit reporting company's inve...,"In late XX/XX/2021, the XXXX XXXX company repo...","EQUIFAX, INC.",Closed with explanation,in late [redacted][redacted]2021 the [redacted...
2506645,Student loan,Dealing with your lender or servicer,as we all know all student loans is not to sta...,"Navient Solutions, LLC.",Closed with explanation,as we all know all student loans is not to sta...
2506647,Debt collection,Written notification about debt,"To Whom it may concern, This letter is to info...","ACIMA CREDIT, LLC",Closed with explanation,to whom it may concern this letter is to infor...
